# Homework Exercise 5: Spark DataFrames

## Task 0

### Homework 5 Completed by Ahsan Ahmad

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

## TASK 1: Display the content of a directory that contains the data

In [0]:
%fs ls "dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/"

path,name,size,modificationTime
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv,2010-summary.csv,7121,1522192049000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2011-summary.csv,2011-summary.csv,7069,1522192049000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2012-summary.csv,2012-summary.csv,6857,1522192049000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2013-summary.csv,2013-summary.csv,7020,1522192050000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2014-summary.csv,2014-summary.csv,6729,1522192050000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv,2015-summary.csv,7080,1522192050000


In [0]:
#1b(i)
# Veiwing directory contents as a table
dbutils.fs.ls("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/")

[FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv', name='2010-summary.csv', size=7121, modificationTime=1522192049000),
 FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2011-summary.csv', name='2011-summary.csv', size=7069, modificationTime=1522192049000),
 FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2012-summary.csv', name='2012-summary.csv', size=6857, modificationTime=1522192049000),
 FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2013-summary.csv', name='2013-summary.csv', size=7020, modificationTime=1522192050000),
 FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2014-summary.csv', name='2014-summary.csv', size=6729, modificationTime=1522192050000),
 FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv', name='2015-summary.csv', size=7080, modificationTime=15221920

In [0]:
#1b(ii)
display(dbutils.fs.ls("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/"))

path,name,size,modificationTime
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv,2010-summary.csv,7121,1522192049000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2011-summary.csv,2011-summary.csv,7069,1522192049000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2012-summary.csv,2012-summary.csv,6857,1522192049000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2013-summary.csv,2013-summary.csv,7020,1522192050000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2014-summary.csv,2014-summary.csv,6729,1522192050000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv,2015-summary.csv,7080,1522192050000


## TASK 2: Read and view schema of two .csv files

In [0]:
#2a

# Reading in the 2015 data in and saving the file as flight2015DF
flight2015DF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv", header = True)

In [0]:
#2b

# Viewing the schema now using printSchema() command
flight2015DF.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)



In [0]:
#2c

# Setting the inferSchema parameter to True and re-reading the 2015 data
flight2015DF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv", header = True, inferSchema = True)

In [0]:
#2d

# Viewing the schema again to show that count is now integer
flight2015DF.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [0]:
#2e

# Reading in the 2014 flight data file and saving it as flight2014DF
flight2014DF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2014-summary.csv", header = True, inferSchema = True)

## TASK 3: Combine two Dataframes and read in all .csv files

In [0]:
#3a

# Combining both the dataframes above into a single one using union() method of Spark DataFrame
flight_dataDF = flight2015DF.union(flight2014DF)

In [0]:
#3b

# Using the count() method to check the new number of rows for the combined dataframe
flight_dataDF.count()

497

In [0]:
#3c

# Reading all the flight data files to a single datframe named flightsAllDF
flightsAllDF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/*.csv", header = True, inferSchema = True)

## TASK 4. Read in all .csv files using a custom schema

In [0]:
#4a

# Creating a custom Schema named myschema
myschema = StructType( [
    StructField("dest", StringType()),
    StructField("origin", StringType()),
    StructField("count", LongType()) ] )

# Saving the file again using schema = myschema instead of inferring schema
flightsAllDF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/*.csv", header = True, schema = myschema)

In [0]:
#4b

# Printing the schema for the updated flightsAllDF
flightsAllDF.printSchema()

root
 |-- dest: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- count: long (nullable = true)



In [0]:
#4c

# Using count() and .distinct().count() to get the count and distinct count of rows for flightsAllDF dataframe
print("count:", flightsAllDF.count())

print("distinct count:", flightsAllDF.distinct().count())

# The difference is due to duplicate rows in the files as we don't have a primary key identifier for these files it is possible that some rows have the same destination and origin airport along with the same number of counts. To mitigate this issue we can add a column for primary key identifier in all these files

count: 1502
distinct count: 1328


## TASK 5. View the DF

In [0]:
#5a

# Using .colect() action command to show all the records of the DataFrame flightsAllDF, expensive operaion
flightsAllDF.collect()

[Row(dest='United States', origin='Romania', count=1),
 Row(dest='United States', origin='Ireland', count=264),
 Row(dest='United States', origin='India', count=69),
 Row(dest='Egypt', origin='United States', count=24),
 Row(dest='Equatorial Guinea', origin='United States', count=1),
 Row(dest='United States', origin='Singapore', count=25),
 Row(dest='United States', origin='Grenada', count=54),
 Row(dest='Costa Rica', origin='United States', count=477),
 Row(dest='Senegal', origin='United States', count=29),
 Row(dest='United States', origin='Marshall Islands', count=44),
 Row(dest='Guyana', origin='United States', count=17),
 Row(dest='United States', origin='Sint Maarten', count=53),
 Row(dest='Malta', origin='United States', count=1),
 Row(dest='Bolivia', origin='United States', count=46),
 Row(dest='Anguilla', origin='United States', count=21),
 Row(dest='Turks and Caicos Islands', origin='United States', count=136),
 Row(dest='United States', origin='Afghanistan', count=2),
 Row(

In [0]:
#5b

# Showing the first three rows of the distributed DF using three different action commands: .show(n=3), .head(n=3) and .take(n=3)

flightsAllDF.show(n=3)

flightsAllDF.head(n=3)

flightsAllDF.take(3)

# .show() gives the output as a table of rows while .head() and .take() gives us the output as first n rows as a list of rows

+-------------+-------+-----+
|         dest| origin|count|
+-------------+-------+-----+
|United States|Romania|    1|
|United States|Ireland|  264|
|United States|  India|   69|
+-------------+-------+-----+
only showing top 3 rows



[Row(dest='United States', origin='Romania', count=1),
 Row(dest='United States', origin='Ireland', count=264),
 Row(dest='United States', origin='India', count=69)]

In [0]:
#5c

# Using the .first() to first save the first row and then using the .type() to print it's type

first_row = flightsAllDF.first()

type(first_row)

pyspark.sql.types.Row

## TASK 6. Obtain a column called filename and rename the count column

In [0]:
#6a

# Renaming the count column to numflights column and creating a 4th column named filename and saving everything in a new dataframe flightsAllDF_1

flightsAllDF_1 = flightsAllDF.withColumnRenamed('count', 'numflights').withColumn('filename', F.input_file_name())


In [0]:
#6b

# Repeating the above task 6a by using .select() instead
flightsAllDF_1 = flightsAllDF.select(flightsAllDF['dest'],flightsAllDF['origin'], flightsAllDF['count'].alias('numflights'), F.input_file_name().alias('filename'))

In [0]:
#6c

# Outputing 5 records with just the numflights and filename columns
flightsAllDF_1.select(flightsAllDF_1['numflights'], flightsAllDF_1['filename']).show(n=5, truncate = False)

+----------+--------------------------------------------------------------------------------+
|numflights|filename                                                                        |
+----------+--------------------------------------------------------------------------------+
|1         |dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv|
|264       |dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv|
|69        |dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv|
|24        |dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv|
|1         |dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv|
+----------+--------------------------------------------------------------------------------+
only showing top 5 rows



In [0]:
#6d

# Saving flightsAllDF_1 as flightsAllDF (overwritting it) and printing out column names
flightsAllDF = flightsAllDF_1

flightsAllDF.columns

['dest', 'origin', 'numflights', 'filename']

## TASK 7. Add a year column extracted from filename

In [0]:
#7a

# Adding a new column 'year' which has the year number extracted from the filename column in it'
flightsAllDF_2 = flightsAllDF.select('*', F.regexp_extract('filename',r'([0-9]{4})',1).alias('year').cast(IntegerType()))

In [0]:
#7b

# Dropping column filename from flightsAllDF_2 and saving it back to flightsAllDF
flightsAllDF = flightsAllDF_2.drop('filename')

In [0]:
#7c

# Viewing the 5 records in table format withh all columns for the dataframe flightsAllDF
flightsAllDF.show(n=5)

+-----------------+-------------+----------+----+
|             dest|       origin|numflights|year|
+-----------------+-------------+----------+----+
|    United States|      Romania|         1|2010|
|    United States|      Ireland|       264|2010|
|    United States|        India|        69|2010|
|            Egypt|United States|        24|2010|
|Equatorial Guinea|United States|         1|2010|
+-----------------+-------------+----------+----+
only showing top 5 rows



In [0]:
#7d

# Viewing the data types of the columns using schema
flightsAllDF.printSchema()

root
 |-- dest: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- numflights: long (nullable = true)
 |-- year: integer (nullable = true)



## TASK 8. Sorting DF

In [0]:
#8a

# Sorting the flightsAllDF by 'dest' in ascending order and 'numflight' in descending order and showing 10 rows
flightsAllDF.sort(['dest','numflights'], ascending = [True, False]).show(n=10)


+-----------+-------------+----------+----+
|       dest|       origin|numflights|year|
+-----------+-------------+----------+----+
|Afghanistan|United States|        11|2010|
|Afghanistan|United States|         8|2011|
|Afghanistan|United States|         5|2012|
|    Algeria|United States|         9|2014|
|    Algeria|United States|         4|2015|
|    Algeria|United States|         2|2013|
|     Angola|United States|        15|2015|
|     Angola|United States|        14|2010|
|     Angola|United States|        13|2011|
|     Angola|United States|        13|2014|
+-----------+-------------+----------+----+
only showing top 10 rows



## TASK 9. Counts of distinct column values

In [0]:
#9a

# Showing the number of distinct destinations in the dataset
flightsAllDF.select(F.countDistinct('dest')).show()

+--------------------+
|count(DISTINCT dest)|
+--------------------+
|                 167|
+--------------------+



In [0]:
#9b

# Showing the many distinct combinations of destination and year
flightsAllDF.select('dest','year').distinct().count()

772

## TASK 10. Flights from USA to Japan or South Korea

In [0]:
#10a

# Number of records that contain either Japan or South Korea as the destination
flightsAllDF.filter((flightsAllDF['dest'] == 'Japan') | (flightsAllDF['dest'] == 'South Korea')).count()

12

In [0]:
#10b

# Number of flights United States send to South Korea in 2013
flightsAllDF.filter((flightsAllDF['dest'] == 'South Korea') & (flightsAllDF['year'] == 2013)).show()

+-----------+-------------+----------+----+
|       dest|       origin|numflights|year|
+-----------+-------------+----------+----+
|South Korea|United States|       842|2013|
+-----------+-------------+----------+----+



## TASK 11. Produce a bar chart of the domestic flights across years

In [0]:
#11a

# Displaying a bar chart of the number of Domestic flights across years, first we will subset the data to get these values only
flightsAllDF.filter((flightsAllDF['dest'] == 'United States') & (flightsAllDF['origin'] == 'United States')).sort('year', ascending = True).display()

dest,origin,numflights,year
United States,United States,348113,2010
United States,United States,352742,2011
United States,United States,347452,2012
United States,United States,343132,2013
United States,United States,358354,2014
United States,United States,370002,2015


Databricks visualization. Run in Databricks to view.

## TASK 12. Counts of total flights

In [0]:
#12a

# Total number of flights flown in each of the 6 years
flightsAllDF.groupby('year').sum('numflights').sort('sum(numflights)',ascending = False).show()

+----+---------------+
|year|sum(numflights)|
+----+---------------+
|2015|         453316|
|2014|         436855|
|2011|         428426|
|2010|         422269|
|2012|         422180|
|2013|         417869|
+----+---------------+



## TASK 13. Counts of International flights

In [0]:
#13a

# Obtaining the total number of international flights that flew to United States and saving it in a seperate dataframe
inUSADF = flightsAllDF.filter((flightsAllDF['origin'] != 'United States') & (flightsAllDF['dest'] == 'United States')).groupby('year').sum('numflights').withColumnRenamed('sum(numflights)', 'totalflightsin').sort('totalflightsin')
inUSADF.show()

+----+--------------+
|year|totalflightsin|
+----+--------------+
|2010|         36819|
|2013|         36830|
|2012|         36890|
|2011|         37763|
|2014|         38833|
|2015|         41350|
+----+--------------+



In [0]:
#13b

# Obtaining the total number of international flights that flew out United States and saving it in a seperate dataframe
outUSADF = flightsAllDF.filter((flightsAllDF['origin'] == 'United States') & (flightsAllDF['dest'] != 'United States')).groupby('year').sum('numflights').withColumnRenamed('sum(numflights)', 'totalflightsout').sort('totalflightsout')
outUSADF.show()

# Joining both inUSADF and outUSADF dataframes by using join() method of Spark Dataframe
joined_USADF = inUSADF.join(outUSADF, on = 'year', how = 'inner')
joined_USADF = joined_USADF.select('*', ((joined_USADF['totalflightsin']) > (joined_USADF['totalflightsout'])).alias('morein')).sort('year')
joined_USADF.show()


+----+---------------+
|year|totalflightsout|
+----+---------------+
|2010|          37337|
|2012|          37838|
|2013|          37907|
|2011|          37921|
|2014|          39668|
|2015|          41964|
+----+---------------+

+----+--------------+---------------+------+
|year|totalflightsin|totalflightsout|morein|
+----+--------------+---------------+------+
|2010|         36819|          37337| false|
|2011|         37763|          37921| false|
|2012|         36890|          37838| false|
|2013|         36830|          37907| false|
|2014|         38833|          39668| false|
|2015|         41350|          41964| false|
+----+--------------+---------------+------+



## TASK 14. Aggregate statistics

In [0]:
#14a

# Filtering the rows to get the records where dest is United States and the origin is not United States and than applying the Aggregate Statistics on the resulting DF to get sum, avg and max for each year
flightsAllDF.filter((flightsAllDF['origin'] != 'United States') & (flightsAllDF['dest'] == 'United States')).groupby('year').agg(F.sum('numflights').alias('sumflightstoUS'), F.avg('numflights').alias('avgflightstoUS'), F.max('numflights').alias('maxflightstoUS')).sort('year').show()

+----+--------------+------------------+--------------+
|year|sumflightstoUS|    avgflightstoUS|maxflightstoUS|
+----+--------------+------------------+--------------+
|2010|         36819| 283.2230769230769|          8305|
|2011|         37763| 299.7063492063492|          8650|
|2012|         36890| 312.6271186440678|          8097|
|2013|         36830| 320.2608695652174|          7983|
|2014|         38833| 331.9059829059829|          8177|
|2015|         41350|333.46774193548384|          8483|
+----+--------------+------------------+--------------+

